In [1]:
import os
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_titles_info(title_id: str):
    title_info_dict = {
        "title": "",
        "overview": "",
        "genres": [],
        "cast": [],
        "content_is": [],
        "subtitles": [],
        "audio": [],
    }

    request_url = f"https://www.netflix.com/title/{title_id}"
    response = requests.get(
        request_url, 
        # verify=False
    )
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    if soup:
        title_h1 = soup.find("h1", class_="title-title")
        if title_h1:
            title_info_dict.update({
                "title": title_h1.text
            })

        overview_div = soup.find("div", class_="title-info-synopsis")
        if overview_div:
            title_info_dict.update({            
                "overview": overview_div.text
            })

        genre_div = soup.find("div", "more-details-cell cell-genres")
        if genre_div:
            title_info_dict.update({
                "genres": [spn.text.replace(',', '').strip() for spn in genre_div.findAll('span')],
            })
            

        cast_div = soup.find("div", "more-details-cell cell-cast")
        if cast_div:
            title_info_dict.update({
                "cast": [spn.text.replace(',', '').strip() for spn in cast_div.findAll('span')]
            })


        content_mood_div = soup.find("div", "more-details-cell cell-mood-tag")
        if content_mood_div:
            title_info_dict.update({
                "content_is": [spn.text.replace(',', '').strip() for spn in content_mood_div.findAll('span')],
            })

        subtitles_div = soup.find("div", "more-details-cell cell-subtitle")
        if subtitles_div:
            title_info_dict.update({
                "subtitles": [spn.text.replace(',', '').strip() for spn in subtitles_div.findAll('span')],
            })

        audio_div = soup.find("div", "more-details-cell cell-audio")
        if subtitles_div:
            title_info_dict.update({
                "audio": [spn.text.replace(',', '').strip() for spn in audio_div.findAll('span')],
            })

    return title_info_dict

In [3]:
def request_netflix_titles(
        params,
        cookies, 
        headers,
        genre_id = "34399",
        from_ix = 0,
        to_ix = 100,
    ):

    genre_item_summ_path = f'["genres",{genre_id},"az",{{"from":{from_ix},"to":{to_ix}}},"itemSummary"]'
    genre_item_ref_summ_path = f'["genres",{genre_id},"az",{{"from":{from_ix},"to":{to_ix}}},"reference",["availability","episodeCount","inRemindMeList","queue","summary"]]'
    data = {
        'path': [
            genre_item_summ_path,
            genre_item_ref_summ_path,
        ]
    }

    response = requests.post(
        'https://www.netflix.com/nq/website/memberapi/vc63e5850/pathEvaluator',
        params=params,
        cookies=cookies,
        headers=headers,
        data=data,
        # verify=False
    )

    print(f"fetch status code --> {response.status_code}")
    return response



In [4]:
def process_titles_data(genre_id, summary_data):
    title_info_list = []

    for ix, item_summ in summary_data["jsonGraph"]["genres"][genre_id]['az'].items():
        title_info_dict = {}

        if not item_summ["itemSummary"].get("value"):
            print("No data is present, stopping here")
            break

        item_summ_val = item_summ["itemSummary"]["value"]

        print(f"processing title {int(ix)+1}: id -> {item_summ_val['id']}")

        misc_title_info_dict = get_titles_info(str(item_summ_val["id"]))

        title_info_dict.update({
            "id": str(item_summ_val["id"]),
            "title": item_summ_val["title"],
            "type": item_summ_val["type"],
            "release_year": item_summ_val["releaseYear"],
            "runtime": item_summ_val.get("infoDensityRuntime"),
            "backdrop_path": item_summ_val["boxArt"]["url"],
            "made_by_netflix": item_summ_val["isOriginal"],
            "maturity_rating": item_summ_val["maturity"]["rating"]["value"],
            "maturity_warning": item_summ_val["maturity"]["rating"]["maturityDescription"],
            "maturity_rating_reason": [ix for ix in item_summ_val["maturity"]["rating"]["specificRatingReason"].split(', ')],
            "maturity_rating_board": item_summ_val["maturity"]["rating"]["board"],
        })

        title_info_dict = {**title_info_dict, **misc_title_info_dict}

        title_info_list.append(title_info_dict)

    return title_info_list


In [7]:
def main():

    from_ix = 3000
    to_ix = 3005
    genre_id = "34399"
    
    params = {
        'webp': 'true',
        'drmSystem': 'widevine',
        'isVolatileBillboardsEnabled': 'true',
        'routeAPIRequestsThroughFTL': 'false',
        'hasVideoMerchInBob': 'true',
        'hasVideoMerchInJaw': 'true',
        'falcor_server': '0.1.0',
        'withSize': 'true',
        'materialize': 'true',
        'original_path': '/shakti/mre/pathEvaluator',
    }

    cookies = {
        'SecureNetflixId': '<your_netflix_id>',
        'NetflixId': '<your_netflix_id>',
        'profilesNewSession': '0'
    }

    headers = {
        'authority': 'www.netflix.com',
        'accept': '*/*',
        'accept-language': 'en-GB,en;q=0.7',
        'content-type': 'application/x-www-form-urlencoded',
    }

    response = request_netflix_titles(
        params=params,
        cookies=cookies, 
        headers=headers, 
        from_ix=from_ix, 
        to_ix=to_ix,
    )
    
    if response.status_code == 200:
        summary_data = response.json()
        title_info_list = process_titles_data(genre_id, summary_data)
        df = pd.DataFrame.from_records(title_info_list)

        if not df.empty:
            csv_foldername = "csv_files"
            if not os.path.isdir(csv_foldername):
                os.makedirs(csv_foldername)
                
            csv_filename = os.path.join(
                csv_foldername, 
                f"netflix_movies_from_{from_ix}_to_{to_ix}.csv"
            )

            with open(f"{csv_filename.replace('.csv', '.json')}", 'w') as fi:
                json.dump(title_info_list, fi)
                
            df.to_csv(csv_filename, index=False)
            print(f"CSV file: {csv_filename} saved successfully!")
            return df
        else:
            print("nothing to save...")
            return
            
    else:
        print("failed to get data. Input correct Netflix Id or check on with correct genre_id and index")

In [8]:
if __name__ == "__main__":
    df = main()

fetch status code --> 200
processing title 3001: id -> 81498297
processing title 3002: id -> 81251628
processing title 3003: id -> 70251897
processing title 3004: id -> 81150305
processing title 3005: id -> 81567391
processing title 3006: id -> 80117263
CSV file: csv_files/netflix_movies_from_3000_to_3005.csv saved successfully!
